In [5]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from subprocess import check_output 
import sys
from datetime import datetime
%matplotlib inline
os.chdir('C:/Kaggle/Airbnb')

In [6]:
countries=pd.read_csv('countries.csv')
sessions=pd.read_csv('sessions.csv')
test_users=pd.read_csv('test_users.csv')
train_users_2=pd.read_csv('train_users_2.csv')
train_lr=pd.read_csv('train_p.csv')
test_lr=pd.read_csv('test_p.csv')

In [54]:
X_train=train_users_2[train_users_2['date_account_created'].str.slice(0,4,1).astype(int)>=2014]
X_test=test_users
X_test['country_destination']="NDF"

In [14]:
X_train['first_browser']=X_train['first_browser'].replace(X_train['first_browser'][X_train['first_browser'].str.contains(r'Chrom',case=False,na=False)].unique().tolist(),"Chrome")
X_train['first_browser'].replace(["Firefox","Mobile Firefox","Mozilla"],'Firefox',inplace=True)
X_train['first_browser'].replace(["IE","IE Mobile"],'IE',inplace=True)
X_train['first_browser'].replace(["Mobile Safari","Safari"],'Safari',inplace=True)
X_train['first_browser'].replace(["Opera","Opera Mini","Opera Mobile"],'Opera',inplace=True)
X_train['first_browser'].replace(["Arora","Avant Browser","Camino","CometBird","Comodo Dragon","Conkeror","CoolNovo","Crazy Browser","Epic","Flock","Google Earth","Googlebot","IBrowse","IceDragon","IceWeasel","Iron","Kindle Browser","Maxthon","Nintendo Browser","NetNewsWire","OmniWeb","Outlook 2007","Pale Moon","Palm Pre web browser","PS Vita browser","RockMelt","SeaMonkey","SiteKiosk","SlimBrowser","Sogou Explorer","Stainless","TenFourFox","TheWorld Browser","UC Browser","wOSBrowser","Yandex.Browser"],'Other',inplace=True)

X_test['first_browser']=X_test['first_browser'].replace(X_test['first_browser'][X_test['first_browser'].str.contains(r'Chrom',case=False,na=False)].unique().tolist(),"Chrome")
X_test['first_browser'].replace(["Firefox","Mobile Firefox","Mozilla"],'Firefox',inplace=True)
X_test['first_browser'].replace(["IE","IE Mobile"],'IE',inplace=True)
X_test['first_browser'].replace(["Mobile Safari","Safari"],'Safari',inplace=True)
X_test['first_browser'].replace(["Opera","Opera Mini","Opera Mobile"],'Opera',inplace=True)
X_test['first_browser'].replace(["Arora","Avant Browser","Camino","CometBird","Comodo Dragon","Conkeror","CoolNovo","Crazy Browser","Epic","Flock","Google Earth","Googlebot","IBrowse","IceDragon","IceWeasel","Iron","Kindle Browser","Maxthon","Nintendo Browser","NetNewsWire","OmniWeb","Outlook 2007","Pale Moon","Palm Pre web browser","PS Vita browser","RockMelt","SeaMonkey","SiteKiosk","SlimBrowser","Sogou Explorer","Stainless","TenFourFox","TheWorld Browser","UC Browser","wOSBrowser","Yandex.Browser"],'Other',inplace=True)


C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
X_train['age'][X_train['age'].isnull()]=-1
X_train['age'][X_train['age']>1800]=2014-X_train['age'][X_train['age']>1800]
X_test['age'][X_test['age'].isnull()]=-1
X_test['age'][X_test['age']>1800]=2014-X_test['age'][X_test['age']>1800]
#X_train['age'].quantile(q=[0,0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99,1])

C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

In [28]:
#Merging LR predictions
train_lr.drop(['Unnamed: 0'],inplace=True,axis=1)
train_lr.columns=['id', 'pred_lr']
test_lr.drop(['Unnamed: 0'],inplace=True,axis=1)
test_lr.columns=['id', 'pred_lr']
X_train=pd.merge(X_train,train_lr,on='id',how='left')
X_test=pd.merge(X_test,test_lr,on='id',how='left')
X_train['pred_lr'][X_train['pred_lr'].isnull()]=-1
X_test['pred_lr'][X_test['pred_lr'].isnull()]=-1

C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
(X_train.pred_lr==-1).sum()

2651

In [252]:
# Sessions data features
sessions=sessions.rename(columns={'user_id':'id'})
sessions=sessions[sessions['id'].isin(X_train['id'].unique().tolist()+X_test['id'].unique().tolist())]
sessions['action'].replace(['agree_terms_check','agree_terms_uncheck'],'agree_terms',inplace=True)
sessions['action'].replace(['ajax_google_translate_description','ajax_google_translate_reviews'],'google_translate',inplace=True)
sessions['action'].replace(['ajax_payout_edit','ajax_payout_options_by_country'],'payout',inplace=True)
sessions['action'].replace(['ajax_photo_widget','ajax_photo_widget_form_iframe'],'photo_widget',inplace=True)
sessions['action'].replace(['ajax_referral_banner_experiment_type','ajax_referral_banner_type'],'banner',inplace=True)
sessions['action'].replace(['create_ach','create_multiple','create_paypal'],'create',inplace=True)
sessions['action'].replace(['complete_redirect','complete_status'],'complete',inplace=True)
sessions['action'].replace(['departments','department'],'department',inplace=True)
sessions['action'].replace(['edit_verification','edit'],'edit',inplace=True)
sessions['action'].replace(['email_itinerary_colorbox','email_share','email_wishlist'],'email',inplace=True)
sessions['action'].replace(['friends_new','friends'],'friends',inplace=True)
sessions['action'].replace(['home_safety_landing','home_safety_terms'],'home_safety',inplace=True)
sessions['action'].replace(['jumio_redirect','jumio_token'],'jumio',inplace=True)
sessions['action'].replace(['kba_update','kba'],'kba',inplace=True)
sessions['action'].replace(['listings','listing'],'listing',inplace=True)
sessions['action'].replace(['message_to_host_change','message_to_host_focus'],'message_to_host',inplace=True)
sessions['action'].replace(['multi_message','multi'],'multi',inplace=True)
sessions['action'].replace(['qt_reply_v2','qt_with','qt2'],'qt',inplace=True)
sessions['action'].replace(['reviews_new','reviews'],'reviews',inplace=True)
sessions['action'].replace(['similar_listings_v2'],'similar_listings',inplace=True)
sessions['action'].replace(['show_code'],'show',inplace=True)
sessions['action'].replace(['social_connections'],'social',inplace=True)
sessions['action'].replace(['terms_and_conditions'],'terms',inplace=True)
sessions['action'].replace(['transaction_history_paginated'],'transaction_history',inplace=True)
sessions['action'].replace(['travel_plans_current','travel_plans_previous'],'travel_plans',inplace=True)
sessions['action'].replace(['update_cached','update_friends_display','update_hide_from_search_engines','update_notifications'],'update',inplace=True)


In [253]:
#sessions.head()
df_action = pd.crosstab(index=sessions['id'], columns=[sessions['action']]).fillna(0)
df_action.reset_index(inplace=True)
df_action.iloc[:,1:]=df_action.iloc[:,1:].clip(0,1)
df_action.columns = ['id']+['action_' + str(col) for col in df_action.columns[1:]]

df_action_type = pd.crosstab(index=sessions['id'], columns=[sessions['action_type']]).fillna(0)
df_action_type.reset_index(inplace=True)
df_action_type.iloc[:,1:]=df_action_type.iloc[:,1:].clip(0,1)
df_action_type.columns = ['id']+['action_type_' + str(col) for col in df_action_type.columns[1:]]

df_action_detail = pd.crosstab(index=sessions['id'], columns=[sessions['action_detail']]).fillna(0)
df_action_detail.reset_index(inplace=True)
df_action_detail.iloc[:,1:]=df_action_detail.iloc[:,1:].clip(0,1)
df_action_detail.columns = ['id']+['action_detail_' + str(col) for col in df_action_detail.columns[1:]]

In [254]:
df_action_detail=df_action_detail.assign(
          action_detail_coupon =df_action_detail.action_detail_apply_coupon+df_action_detail.action_detail_apply_coupon_click+df_action_detail.action_detail_apply_coupon_click_success+df_action_detail.action_detail_apply_coupon_error+df_action_detail.action_detail_coupon_code_click+df_action_detail.action_detail_coupon_field_focus,
          action_detail_cancel =df_action_detail.action_detail_cancellation_policies+df_action_detail.action_detail_cancellation_policy+df_action_detail.action_detail_cancellation_policy_click,
          action_detail_guest  =df_action_detail.action_detail_guest_cancellation+df_action_detail.action_detail_guest_itinerary+df_action_detail.action_detail_guest_receipt,
          action_detail_host   =df_action_detail.action_detail_host_guarantee+df_action_detail.action_detail_host_home+df_action_detail.action_detail_host_refund_guest+df_action_detail.action_detail_host_respond+df_action_detail.action_detail_host_respond_page+df_action_detail.action_detail_host_standard_suspension,
          action_detail_listing=df_action_detail.action_detail_listing_descriptions+df_action_detail.action_detail_listing_recommendations+df_action_detail.action_detail_listing_reviews+df_action_detail.action_detail_listing_reviews_page+df_action_detail.action_detail_manage_listing+df_action_detail.action_detail_view_listing,
          action_detail_message=df_action_detail.action_detail_lookup_message_thread+df_action_detail.action_detail_message_inbox+df_action_detail.action_detail_message_post+df_action_detail.action_detail_message_thread+df_action_detail.action_detail_message_to_host_change+df_action_detail.action_detail_message_to_host_focus,
                                        )
df_action_detail.iloc[:,1:]=df_action_detail.iloc[:,1:].clip(0,1)

In [276]:
X_train=pd.merge(X_train,df_action,on='id',how='left')
X_train=pd.merge(X_train,df_action_type,on='id',how='left')
X_train=pd.merge(X_train,df_action_detail,on='id',how='left')
X_train.loc[:,X_train.loc[:,X_train.columns.to_series().str.startswith('action').tolist()].columns.tolist()]=X_train.loc[:,X_train.loc[:,X_train.columns.to_series().str.startswith('action').tolist()].columns.tolist()].replace(np.nan,0)

X_test=pd.merge(X_test,df_action,on='id',how='left')
X_test=pd.merge(X_test,df_action_type,on='id',how='left')
X_test=pd.merge(X_test,df_action_detail,on='id',how='left')
X_test.loc[:,X_test.loc[:,X_test.columns.to_series().str.startswith('action').tolist()].columns.tolist()]=X_test.loc[:,X_test.loc[:,X_test.columns.to_series().str.startswith('action').tolist()].columns.tolist()].replace(np.nan,0)

In [64]:
train_ids=X_train['id']
test_ids=X_test['id']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(X_train['country_destination'].unique().tolist())
target=le.transform(X_train['country_destination'])

train_id_target=pd.DataFrame({'target':target.tolist(),'target_map':le.inverse_transform(target).tolist()})
train_id_target.to_pickle('train_id_target')
pd.DataFrame(test_ids).to_pickle('test_ids')



In [372]:
#cp=X_train.copy()
X_train=X_train.assign(year_account=X_train['date_account_created'].str.slice(0,4,1).astype(int),
                       month_account=X_train['date_account_created'].str.slice(5,7,1).astype(int),
                       day_account=X_train['date_account_created'].str.slice(8,10,1).astype(int),
                       weekday_account=pd.to_datetime(X_train['date_account_created'],format='%Y-%m-%d').dt.dayofweek,
                       year_active=X_train['timestamp_first_active'].astype(str).str.slice(0,4,1).astype(int),
                       month_active=X_train['timestamp_first_active'].astype(str).str.slice(4,6,1).astype(int),
                       day_active=X_train['timestamp_first_active'].astype(str).str.slice(6,8,1).astype(int),
                       weekday_active=pd.to_datetime(((X_train.timestamp_first_active).astype(str).str.slice(0,4,1)+ '-' + (X_train.timestamp_first_active).astype(str).str.slice(4,6,1) + '-' + (X_train.timestamp_first_active).astype(str).str.slice(6,8,1)),format='%Y-%m-%d').dt.dayofweek,
                      )
X_test=X_test.assign(year_account=X_test['date_account_created'].str.slice(0,4,1).astype(int),
                       month_account=X_test['date_account_created'].str.slice(5,7,1).astype(int),
                       day_account=X_test['date_account_created'].str.slice(8,10,1).astype(int),
                       weekday_account=pd.to_datetime(X_test['date_account_created'],format='%Y-%m-%d').dt.dayofweek,
                       year_active=X_test['timestamp_first_active'].astype(str).str.slice(0,4,1).astype(int),
                       month_active=X_test['timestamp_first_active'].astype(str).str.slice(4,6,1).astype(int),
                       day_active=X_test['timestamp_first_active'].astype(str).str.slice(6,8,1).astype(int),
                       weekday_active=pd.to_datetime(((X_test.timestamp_first_active).astype(str).str.slice(0,4,1)+ '-' + (X_test.timestamp_first_active).astype(str).str.slice(4,6,1) + '-' + (X_test.timestamp_first_active).astype(str).str.slice(6,8,1)),format='%Y-%m-%d').dt.dayofweek,
                      )
X_train.drop(['id', 'date_first_booking','country_destination','date_account_created','timestamp_first_active'], axis=1, inplace=True)
X_test.drop(['id', 'date_first_booking','country_destination','date_account_created','timestamp_first_active'], axis=1, inplace=True)
X_train['age']=X_train['age'].map(lambda x: -1 if x<15 or x>100 else x)

In [432]:
#X_train.loc[:,list(~np.array(X_train.columns.str.contains('action').tolist()))].columns.tolist()

In [464]:
X_train['signup_flow']='signup_flow'+ X_train['signup_flow'].astype(str)
X_train['weekday_account']='weekday_account'+ X_train['weekday_account'].astype(str)
X_train['weekday_active']='weekday_active'+ X_train['weekday_active'].astype(str)
X_test['signup_flow']='signup_flow'+ X_test['signup_flow'].astype(str)
X_test['weekday_account']='weekday_account'+ X_test['weekday_account'].astype(str)
X_test['weekday_active']='weekday_active'+ X_test['weekday_active'].astype(str)
tot=X_train.append(X_test)
enc_var= ['gender','signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked','signup_app','first_device_type','first_browser','weekday_active','weekday_account']
onehot=pd.get_dummies(tot[enc_var])
tot=pd.concat([tot.drop(enc_var,axis=1),onehot],axis=1)
X_train=tot.iloc[:len(X_train),:]
X_test=tot.iloc[len(X_train):,:]

X_train.fillna(0,inplace=True)
X_test.fillna(0,inplace=True)

C:\Users\nitin.jain\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [511]:
#cptrain=X_train.copy()
#cptest=X_test.copy()

#pd.DataFrame(X_train1.describe()).to_csv('xgbdataset1.csv')
#removing binary vars whose sum is less than 5% of length  

#X_train1=X_train.drop(list(set(X_train.loc[:,X_train.sum()<0.01*len(X_train)].columns) & set(X_train.loc[:,X_train.apply(pd.Series.nunique)<=2].columns)),axis=1)
#X_test1=X_test.drop(list(set(X_train.loc[:,X_train.sum()<0.01*len(X_train)].columns) & set(X_train.loc[:,X_train.apply(pd.Series.nunique)<=2].columns)),axis=1)
X_train1.to_pickle('X_train1')
X_test1.to_pickle('X_test1')

In [65]:
#X_train1.head()
#X_train['weekday_account_weekday_account6'].value_counts().apply(lambda x: x*100/len(X_train))

In [3]:
#import xgboost as xgb